In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras 
import glob as gl
import os

# CREACIÓN DATASET FINAL

## Lectura datasets

In [ ]:
datetime = "2011-10-28_11"
path = "src/results/"
dir = "dataset_exp_" + datetime

dfs = []

for root, dirs, files in os.walk(path+dir):
    for file in files:
        if file.endswith(".csv") and file.startswith("dataset"):
            file_path = os.path.join(root, file)
            dfs.append(pd.read_csv(file_path, low_memory=False))

df = pd.concat(dfs, ignore_index=True)
df.count()

datetime          160920
overflow          160920
cap               160920
load              160920
dist              160920
origen_id         160920
dest_id           160920
origen_tag        160920
len_origen_tag    160920
dest_tag          160920
len_dest_tag      160920
modelo            160920
criterion         160920
degree            160920
total_balance     160920
abs_flux          160920
dtype: int64

In [ ]:
df.head()

,datetime,overflow,cap,load,dist,origen_id,dest_id,origen_tag,len_origen_tag,dest_tag,len_dest_tag,modelo,criterion,degree,total_balance,abs_flux
0,2011-10-28 11:00:00,0,40.0,2.161344,443,18,7,80.10.1.4.6.11.28.65.98,9,80.10.1.4.6.11.28.65,8,barabasi,0,2,39.69592,445.896059
1,2011-10-28 11:00:00,0,74.0,1.103378,412,5,23,80.10.1.4.6.11.28.34,8,80.10.1.4.6.11.28,7,barabasi,0,2,39.69592,445.896059
2,2011-10-28 11:00:00,0,74.0,2.107634,749,7,23,80.10.1.4.6.11.28.63,8,80.10.1.4.6.11.28,7,barabasi,0,2,39.69592,445.896059
3,2011-10-28 11:00:00,0,74.0,1.920475,454,11,16,80.10.1.4.20.26.36.64,8,80.10.1.4.20.26.36,7,barabasi,0,2,39.69592,445.896059
4,2011-10-28 11:00:00,0,21.2,4.265903,169,7,23,80.10.1.4.6.11.28.65,8,80.10.1.4.6.11.28,7,barabasi,0,2,39.69592,445.896059


### Comprobación % overflow

In [ ]:
print(str((df['overflow'].value_counts(normalize=True) * 100).get(1, 0)) + "%")

3.535918468804375%


## Combinación con datos de cada nodo

In [ ]:
sim_file = "src/data/simtests/test_" + datetime + ".csv"

df_sim = pd.read_csv(sim_file)
df_sim.head()

,iid,timestamp,datetime,h,Beam Irradiance (W/m2),Diffuse Irradiance (W/m2),Ambient Temperature (C),Plane of Array Irradiance (W/m2),Cell Temperature (C),DC Array Output (W),Pavg,dif
0,1,2011-10-28 11:00:00,2011-10-28,11,618,138,18.7,688.821,31.38,2304.117,65.582476,2238.534524
1,2,2011-10-28 11:00:00,2011-10-28,11,618,138,18.7,688.821,31.38,2304.117,281.536466,2022.580534
2,3,2011-10-28 11:00:00,2011-10-28,11,618,138,18.7,688.821,31.38,2304.117,474.326048,1829.790952
3,4,2011-10-28 11:00:00,2011-10-28,11,618,138,18.7,688.821,31.38,2304.117,273.648567,2030.468433
4,5,2011-10-28 11:00:00,2011-10-28,11,618,138,18.7,688.821,31.38,2304.117,1200.738810,1103.378190


In [ ]:
merged_df = pd.merge(df, df_sim, left_on='origen_id', right_on='iid', how='outer') #se combinan en función de la columna origen_id
merged_df.head()

,datetime_x,overflow,cap,load,dist,origen_id,dest_id,origen_tag,len_origen_tag,dest_tag,...,datetime_y,h,Beam Irradiance (W/m2),Diffuse Irradiance (W/m2),Ambient Temperature (C),Plane of Array Irradiance (W/m2),Cell Temperature (C),DC Array Output (W),Pavg,dif
0,2011-10-28 11:00:00,0,40.0,2.161344,443,18,7,80.10.1.4.6.11.28.65.98,9,80.10.1.4.6.11.28.65,...,2011-10-28,11,618,138,18.7,688.821,31.38,2304.117,142.773288,2161.343712
1,2011-10-28 11:00:00,0,74.0,2.161344,186,18,9,80.10.1.4.6.61.76,7,80.10.1.4.6.61,...,2011-10-28,11,618,138,18.7,688.821,31.38,2304.117,142.773288,2161.343712
2,2011-10-28 11:00:00,0,74.0,2.161344,518,18,9,80.10.1.4.6.61.84,7,80.10.1.4.6.61,...,2011-10-28,11,618,138,18.7,688.821,31.38,2304.117,142.773288,2161.343712
3,2011-10-28 11:00:00,0,21.2,2.161344,692,18,1,80.10.1.4.30.37,6,80.10.1.4.30,...,2011-10-28,11,618,138,18.7,688.821,31.38,2304.117,142.773288,2161.343712
4,2011-10-28 11:00:00,0,21.2,3.982753,633,18,21,80.10.1.0.14,5,80.10.1.0,...,2011-10-28,11,618,138,18.7,688.821,31.38,2304.117,142.773288,2161.343712


In [ ]:
dropcols = ['datetime_x', 'iid', 'origen_tag', 'dest_tag'] #si luego se necesitaran los tags quitar esto
merged_df = merged_df.drop(dropcols, axis=1)
merged_df = merged_df.rename(columns={'datetime_y': 'datetime'})
merged_df.head()

,overflow,cap,load,dist,origen_id,dest_id,len_origen_tag,len_dest_tag,modelo,criterion,...,datetime,h,Beam Irradiance (W/m2),Diffuse Irradiance (W/m2),Ambient Temperature (C),Plane of Array Irradiance (W/m2),Cell Temperature (C),DC Array Output (W),Pavg,dif
0,0,40.0,2.161344,443,18,7,9,8,barabasi,0,...,2011-10-28,11,618,138,18.7,688.821,31.38,2304.117,142.773288,2161.343712
1,0,74.0,2.161344,186,18,9,7,6,barabasi,0,...,2011-10-28,11,618,138,18.7,688.821,31.38,2304.117,142.773288,2161.343712
2,0,74.0,2.161344,518,18,9,7,6,barabasi,0,...,2011-10-28,11,618,138,18.7,688.821,31.38,2304.117,142.773288,2161.343712
3,0,21.2,2.161344,692,18,1,6,5,barabasi,0,...,2011-10-28,11,618,138,18.7,688.821,31.38,2304.117,142.773288,2161.343712
4,0,21.2,3.982753,633,18,21,5,4,barabasi,0,...,2011-10-28,11,618,138,18.7,688.821,31.38,2304.117,142.773288,2161.343712


In [ ]:
merged_df.to_csv(path + datetime + '.csv', index=False)